# Logging and debugging

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker import TrainingJob, Model, Endpoint, EndpointConfig

sess = sagemaker.Session()
sess.set_log(True) # This ensure it logs all the API calls are logged (like trace) as INFO and failure as ERROR. Additionally we will log all the serialization/deserialization of the request/response.
sess.set_debug(True) # This ensure we will poll for updates in cloudwatch logs when the wait call is made. This ensure the customer can monitor the progress of the resource when they wait for the resource to be created/in-service.

# create a training job
training_job = TrainingJob.create(
    training_job_name='xgboost-iris',
    hyper_parameters = HyperParameters(
        objective='multi:softmax',
        num_class='3',
        num_round='10',
        eval_metric='merror'
    ),
    algorithm_specification = AlgorithmSpecification(
        training_image='811284229777.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
        training_input_mode='File'
    ),
    role_arn='arn:aws:iam::xxxxxxxxxxxx:role/service-role/AmazonSageMaker-ExecutionRole-20181029T134365', # replace with execution role
    input_data_config = [
        Channel(
            channel_name='train',
            data_source=DataSource(
                s3_data_source=S3DataSource(
                    s3_data_type='S3Prefix',
                    s3_uri='s3://sagemaker-us-west-2-xxxxxxxxxxxx/iris/train',
                    s3_data_distribution_type='FullyReplicated'
                )
            ),
            compression_type='None',
            record_wrapper_type='None'
        )
    ],
    output_data_config = OutputDataConfig(
        s3_output_path='s3://sagemaker-us-west-2-xxxxxxxxxxxx/iris/output'
    ),
    resource_config = ResourceConfig(
        instance_type='ml.m4.xlarge',
        instance_count=1,
        volume_size_in_gb=30
    ),
    stopping_condition = StoppingCondition(
        max_runtime_in_seconds=600
    ),
    session=sess, # we can improve passing session and region automatically if it is running in studio notebook - customers can pass on this.
    region=sess.boto_region_name  # we can improve passing session and region automatically if it is running in studio notebook - customers can pass on this.
)

# wait for the training job to complete
training_job.wait() # wait is an object method that will wait for the training job to complete and only supported in empheral workload

'''
or customer can do 

while training_job.training_job_status != 'Completed':
    time.sleep(300)
    training_job.refresh()
'''

model = Model.create(
    model_name='xgboost-iris',
    primary_container=Container(
        image='811284229777.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
        model_data_url=training_job.model_artifacts.s3_model_artifacts # here we are getting model data from the training job 
    ),
    execution_role_arn='arn:aws:iam::xxxxxxxxxxxx:role/service-role/AmazonSageMaker-ExecutionRole-20181029T134365',
)
'''
if the model is already created then 
we can use Model.get() to retrive the model
'''

endpoint_config = EndpointConfig.create(
    endpoint_config_name='xgboost-iris',
    production_variants=[
        ProductionVariant(
            variant_name='xgboost-iris',
            model_name=model, # note we can chain it to get the name automatically
            initial_instance_count=1,
            instance_type='ml.m4.xlarge'
        )
    ],
)

endpoint = Endpoint.create(
    endpoint_name='xgboost-iris',
    endpoint_config_name=endpoint_config # note we can chain it to get the name automatically
)

# wait for the endpoint to be in service
endpoint.wait_for_status('InService')

endpoint.invoke(body=b'{"data": [5.1, 3.5, 1.4, 0.2]}', # note the invoke is part of the endpoint object. 
                content_type='application/json',
                accept='application/json')

endpoint.delete()

# Output format

In [ ]:
class TrainingJob:
    def __init__(self, training_job_name, hyper_parameters, algorithm_specification, role_arn, input_data_config, output_data_config, resource_config, stopping_condition, session, region):
        self.training_job_name = training_job_name
        self.hyper_parameters = hyper_parameters
        self.algorithm_specification = algorithm_specification
        self.role_arn = role_arn
        self.input_data_config = input_data_config
        self.output_data_config = output_data_config
        self.resource_config = resource_config
        self.stopping_condition = stopping_condition
        self.session = session
        self.region = region
        self.training_job_status = None

    @classmethod
    def create(self, cls, training_job_name, hyper_parameters, algorithm_specification, role_arn, input_data_config, output_data_config, resource_config, stopping_condition, session, region):
        # create the training job
        pass

    def wait(self):
        # wait for the training job to complete
        pass

    def refresh(self):
        # refresh the training job status
        pass

    def _repr_pretty(self):
        # pretty print the training job
        pass

    def _repr_json(self):
        # return json object of the training job
        pass

    def _repr_yaml(self):
        # return yaml object of the training job
        pass